## Задача 3 (Python/R): Подготовка к встрече с новым владельцем сети магазинов


### Описание задачи



Завтра состоится встреча с новым владельцами бизнеса. Тебе предстоит подготовить информацию для встречи, где были бы собраны ключевые метрики, графики и сегменты на твой взгляд.

Ты можешь использовать любые данные из задачи 2, любые библиотеки и методы, которые посчитаете необходимыми. Решение предлагается сделать в свободной форме, все файлы результата и реализации выложить в GoogleDrive, ссылку прикрепить в GoogleForms (не забудь предоставить доступы). Единственное ограничение - код должен быть написан на Python или R.

У Product Ownerа есть желание понять, сколько единиц каждого товара необходимо поставить в каждый из магазинов в следующем месяце. Целью является совершить одну поставку товара в начале месяца и не довозить его больше, до начала следующего.

Ты можешь использовать любые данные из задачи 2, любые библиотеки и методы, которые посчитаете необходимыми. Решение предлагается сделать в свободной форме, все файлы результата и реализации выложить в GoogleDrive, ссылку прикрепить в GoogleForms (не забудь предоставить доступы). Единственное ограничение - код должен быть написан на Python или R.


### Описание подхода


Выполняем sql запрос, который вернет нам все заказы с их месяцем оформления, описанием товара и названием магазина


Описанием товара выбрана комбинация названия, цвета и размер для полноты данных


Полученный DataFrame иллюстрируем, заполняя месяца, в которых не отмечены заказы по определенному типу товара, суммой заказов 0.


Для удобства просмотра данных и экономии памяти на устройстве проверяющего использую выведение графиков в ipynb. Это оправдано большим количеством графиков, **скрипт удобно остановить, дабы не отображать все графики, ожидая окончания скрипта,** и не хранить их все огромным набором файлов, как это следовало бы делать на практике при формировании презентации.


Это вызвано огромным набором товаров, для упрощения товары были разделены лишь по названию и цвету, данные объединялись вне зависимости от размеров, при необходимости их можно включить в анализ данных за минимальное время.






### Программная реализация


In [1]:
# Необходимые библиотеки
%pip install pandasql sqlalchemy==1.4.46 seaborn


Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from pandasql import sqldf
import seaborn as sns
import matplotlib.pyplot as plt

Загрузка данных

In [3]:
import numpy as np
pd.set_option('display.max_colwidth', None)  # or 199

orders = pd.read_csv('orders.csv')
sales = pd.read_csv('sales.csv')
user = pd.read_csv('user.csv')
store = pd.read_csv('store.csv')
product = pd.read_csv('product.csv')
# fixed the data, got malformed JSON other way
orders['product_info'] = orders['product_info'].str.replace('\'', '\"')
orders['product_info'] = orders['product_info'].str.replace('False', 'false')
orders['product_info'] = orders['product_info'].str.replace('True', 'true')

Считываем SQL запросом набор необходимых нам данных

In [6]:
with open('result_task_3.txt', 'r') as file:
    # Читаем содержимое файла
    query = file.read()

# Сохраняем результат SQL запроса в pandas DataFrame
result_task_3 = sqldf(query, globals())
result_task_3

,store_id,store_name,product_characteristics,amount_by_month,month
0,6,Rio Leninsky,Astral Ascent Anorak Black,82,01
1,6,Rio Leninsky,Astral Ascent Anorak Blue,138,01
2,6,Rio Leninsky,Astral Ascent Anorak Brown,126,01
3,6,Rio Leninsky,Astral Ascent Anorak Green,97,01
4,6,Rio Leninsky,Astral Ascent Anorak Orange,93,01
...,...,...,...,...,...
7435,15,Koltso,Zenith Zephyr Zip-Up Pink,64,12
7436,5,Kapitoly Belyaevo,Zenith Zephyr Zip-Up Purple,55,12
7437,6,Rio Leninsky,Zenith Zephyr Zip-Up Red,65,12
7438,2,Afimoll,Zenith Zephyr Zip-Up White,52,12


Вывод графиков

**(если ждать выведения всех товаров, то это смертельно долго)**

In [ ]:

products = result_task_3['product_characteristics'].unique()
stores = result_task_3['store_name'].unique()

for s in stores:
    for p in products:
        mask = (result_task_3['product_characteristics'] == p) & (result_task_3['store_name'] == s)
        empty_months = set(range(1,13)) - set(result_task_3[mask]['order_month'].unique().astype(int))
        
        product_id = result_task_3[result_task_3['product_characteristics'] == p]['product_id']
        ass = type(product_id)
        product_id = product_id.iloc[0]
        store_id = result_task_3[result_task_3['store_name'] == s]['store_id'][0]
        # now we add empty months into data
        for empty_month in empty_months:
            df = pd.DataFrame([[product_id, store_id, p, s, 0, empty_month]], columns=result_task_3.columns)
            result_task_3 = pd.concat([df, result_task_3],
                                      ignore_index=True)
        
        mask = (result_task_3['product_characteristics'] == p) & (result_task_3['store_name'] == s)
        ax = sns.lineplot(data=result_task_3[mask], x='order_month', y='amount_by_month', hue='product_color')
        ax.set(xlabel='months', ylabel='n_sold', title=p + ' in ' + s)
        plt.show()

### Итоги

Полученные графики оптимальны для предсказания необходимых ежемесячных закупок товаров для каждого отдельного магазина. 

Полученные данные не распостраняются назад дальше года, поэтому такой простой метод достаточен.

Однако при большем периоде исследуемых данных для прогноза следует использовать модели SARIMA/SARIMAX.